<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 2: Preparing Data

## Handling Missing Data and Imputing Values

### Lesson Objectives 

-	After completing this lesson, you should be able to: 
- Drop records according to different criteria
-	Fill missing data according to different criteria
-	Drop duplicate records


## DataFrame NA Functions 

-	The `na` method of DataFrames provides functionality for working with missing data 
- Returns an instance of `DataFrameNAFunctions`
-	The following methods are available: 
  -	`drop`, for dropping rows containing NaN or null values 
  -	`fill`, for replacing NaN or null values 
  -	`replace`,  for replacing values matching specified keys

In [ ]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

In [ ]:
val df = spark.range(0, 10).select("id").
    withColumn("uniform", rand(10L)).withColumn("normal", randn(10L))

val halfTonNaN = udf[Double, Double] (x => if (x > 0.5) Double.NaN else x)

val oneToNaN = udf[Double, Double] (x => if (x > 1.0) Double.NaN else x) 

val dfnan = df.withColumn("nanUniform", halfTonNaN(df("uniform"))).
    withColumn("nanNormal", oneToNaN(df("normal"))).drop("uniform"). 
    withColumnRenamed("nanUniform", "uniform").drop("normal"). 
    withColumnRenamed("nanNoemal", "normal")

dfnan.show()

## DataFrame NA Functions - drop 

-	`drop` is used for dropping rows containing `NaN` or `null` values according to a criteria 
-	Several implementations available:
  -	`drop(minNonNulls, cols)`
  -	`drop(minNonNulls)`
  -	`drop(how,cols)`
  - `drop(cols)`
  -	`drop(how)`
  -	`drop()`
-	`cols` is an `Array` or `Seq` of column names
- how should be equal any or all

In [ ]:
// Dropping Rows With minNonNulls Argument 
dfnan.na.drop(minNonNulls = 3).show()

In [ ]:
// Dropping Rows With How Argument 
dfnan.na.drop("all", Array("uniform", "nanNormal")).show()

In [ ]:
// Dropping Rows With How Argument 
dfnan.na.drop("any", Array("uniform", "nanNormal")).show()

## DataFrame NA Functions - fill 

-	`fill` is used for replacing NaN or null values according to a criteria
-	Several implementations available:
  - `fill(valueMap)`
  -	`fill(value,cols)`
  -	`fill(value)`

In [ ]:
// Filling Missing Data By Column Type
dfnan.na.fill(0.0).show()   

// Filling Missing Data With Column Defaults 
val uniformMean = dfnan.filter("uniform <> 'NaN'").groupBy().agg(mean("uniform")).first()(0)

dfnan.na.fill(Map("uniform" -> uniformMean)).show(5)

// Filling Missing Data With Column Defaults 
val dfCols = dfnan.columns.drop(1)

val dfMeans = dfnan.na.drop().groupBy().
agg(mean("uniform"), mean("nanNormal")).first().toSeq

val meansMap = (dfCols.zip(dfMeans)).toMap
dfnan.na.fill(meansMap).show(5)

## DataFrame NA Functions - replace 

-	`replace` is used for replacing values matching specified keys
-	`cols` argument may be a single column name or an array
-	replacement argument is a map: 
  -	`key` is the value to be matched 
  -	`value` is the replacement value itself

In [ ]:
//Replacing Values in a DataFrame 

dfnan.na.replace("uniform", Map(Double.NaN -> 0.0)).show()

## Duplicates

-	`dropDuplicates` is a `DataFrame` method 
-	Used to remove duplicate rows
-	May specify a subset of columns to check for duplicates

In [ ]:
// Dropping Duplicate Rows 
val dfDuplicates = df.unionAll(sc.parallelize(Seq((10,1,1),(11,1,1))).toDF())

// Dropping Duplicate Rows 
val dfCols = dfnan.withColumnRenamed("nanNormal", "normal").columns

dfDuplicates.dropDuplicates(dfCols).show()

## Lesson Summary

-	Having completed this lesson, you should be able to: 
- Drop records according to different criteria
-	Fill missing data according to different criteria
-	Drop duplicate records

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.